## DrugCell Code in the Notebook

In [49]:
import sys
import os
import numpy as np
import pandas as pd
import torch
import torch.utils.data as du
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import argparse
from torch import optim
import numpy as np
import time
import sklearn
import networkx as nx
import networkx.algorithms.components.connected as nxacc
import networkx.algorithms.dag as nxadag
import torch.utils.data as du
from torch.autograd import Variable
from time import time
import candle
import logging
from torchmetrics.functional import mean_absolute_error
from scipy.stats import spearmanr
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import gc
torch.cuda.empty_cache()
#del variables
gc.collect()
#print("hello")
%matplotlib inline

DEBUG:matplotlib.pyplot:Loaded backend module://matplotlib_inline.backend_inline version unknown.


## ANL INPUTS

In [50]:
model_name='DrugCell'
data_url="http://drugcell.ucsd.edu/downloads/data.tgz"
original_data="data.tgz"
data_predict='drugcell_test.txt'
data_model='drugcell_v1.pt'
predict_url = 'http://drugcell.ucsd.edu/downloads/drugcell_all.txt'
model_url = 'http://drugcell.ucsd.edu/downloads/drugcell_v1.pt'
CUDA_ID = 0
prebuilt_load = "Data/drugcell_v1.pt"
data_dir = "data/"
train_data_file = "Data_Curation_final/Curated_CCLE_Multiomics_files/DrugCell_ANL_inputs/temp_train.txt"
test_data_file = "Data_Curation_final/Curated_CCLE_Multiomics_files/DrugCell_ANL_inputs/temp_test.txt"
val_data_file = "Data_Curation_final/Curated_CCLE_Multiomics_files/DrugCell_ANL_inputs/temp_val.txt"
onto = "Data_Curation_final/Curated_CCLE_Multiomics_files/DrugCell_ANL_inputs/drugcell_ont.txt"
hidden='NOTEBOOK/MODEL/Hidden/'
result='NOTEBOOK/MODEL/Result/'
learning_rate = 0.001
batch_size = 1000
eps=0.00001
genotype_hiddens = 6
drug_hiddens='100,50,6'
final_hiddens=6
genotype="Data_Curation_final/Curated_CCLE_Multiomics_files/DrugCell_ANL_inputs/cell2mutation.txt"
fingerprint='Data_Curation_final/Curated_CCLE_Multiomics_files/DrugCell_ANL_inputs/drug2fingerpint.txt'
cell2id='Data_Curation_final/Curated_CCLE_Multiomics_files/DrugCell_ANL_inputs/cell2ind.txt'
drug2id='Data_Curation_final/Curated_CCLE_Multiomics_files/DrugCell_ANL_inputs/drug2ind.txt'
gene2id='Data_Curation_final/Curated_CCLE_Multiomics_files/DrugCell_ANL_inputs/gene2ind.txt'
output_dir = "NOTEBOOK"
epochs=200
weight_decay=1e-3
iteration="lr_0.001_wd_0.001"
#decay_rate = learning_rate / epochs
#optimizer = "adam"
loss = "mse"

## INPUTS

In [51]:
#model_name='DrugCell'
#data_url="http://drugcell.ucsd.edu/downloads/data.tgz"
#original_data="data.tgz"
#data_predict='drugcell_test.txt'
#data_model='drugcell_v1.pt'
#predict_url = 'http://drugcell.ucsd.edu/downloads/drugcell_all.txt'
#model_url = 'http://drugcell.ucsd.edu/downloads/drugcell_v1.pt'
#CUDA_ID = 0
#prebuilt_load = "Data/drugcell_v1.pt"
#data_dir = "data/"
#train_data_file = "data/drugcell_train.txt"
#test_data_file = "data/drugcell_test.txt"
#val_data_file = "data/drugcell_val.txt"
#onto = "data/drugcell_ont.txt"
#hidden='NOTEBOOK/MODEL/Hidden/'
#result='NOTEBOOK/MODEL/Result/'
#learning_rate = 0.001
#batch_size = 1000
#eps=0.00001
#genotype_hiddens = 6
#drug_hiddens='100,50,6'
#final_hiddens=6
#genotype="data/cell2mutation.txt"
#fingerprint='data/drug2fingerprint.txt'
#cell2id='data/cell2ind.txt'
#drug2id='data/drug2ind.txt'
#gene2id='data/gene2ind.txt'
#output_dir = "NOTEBOOK"
#epochs=200
#weight_decay=1e-3
#iteration="lr_0.001_wd_0.001"
#decay_rate = learning_rate / epochs
#optimizer = "adam"
#loss = "mse"

## UTILS CODE

In [52]:
def pearson_corr(x, y):
    xx = x - torch.mean(x)
    yy = y - torch.mean(y)
    return torch.sum(xx*yy) / (torch.norm(xx, 2)*torch.norm(yy,2))

def calc_pcc(x, y):
    xx = x - torch.mean(x)
    yy = y - torch.mean(y)
    return torch.sum(xx*yy) / (torch.norm(xx, 2)*torch.norm(yy,2))


def calc_mae(y_true, y_pred):
    return sklearn.metrics.mean_absolute_error(y_true=y_true, y_pred=y_pred)

def calc_r2(y_true, y_pred):
    target_mean = torch.mean(y_pred)
    ss_tot = torch.sum((y_pred - target_mean) ** 2)
    ss_res = torch.sum((y_pred - y_true) ** 2)
    r2 = 1 - ss_res / ss_tot
    return r2

def load_ontology(file_name, gene2id_mapping):
    dG = nx.DiGraph()
    term_direct_gene_map = {}
    term_size_map = {}
    file_handle = open(file_name)
    gene_set = set()

    for line in file_handle:
        line = line.rstrip().split()
        if line[2] == 'default':
            dG.add_edge(line[0], line[1])
        else:
            if line[1] not in gene2id_mapping:
                continue

            if line[0] not in term_direct_gene_map:
                term_direct_gene_map[ line[0] ] = set()

            term_direct_gene_map[line[0]].add(gene2id_mapping[line[1]])
            gene_set.add(line[1])

    file_handle.close()
    
    print('There are', len(gene_set), 'genes')
    for term in dG.nodes():
        term_gene_set = set()
        if term in term_direct_gene_map:
            term_gene_set = term_direct_gene_map[term]

        deslist = nxadag.descendants(dG, term)

        for child in deslist:
            if child in term_direct_gene_map:
                term_gene_set = term_gene_set | term_direct_gene_map[child]

        # jisoo
        if len(term_gene_set) == 0:
            print('There is empty terms, please delete term:', term)
            sys.exit(1)
        else:
            term_size_map[term] = len(term_gene_set)

    leaves = [n for n in dG.nodes if dG.in_degree(n) == 0]
    print(leaves)
    #leaves = [n for n,d in dG.in_degree() if d==0]

    uG = dG.to_undirected()
    connected_subG_list = list(nxacc.connected_components(uG))

    print('There are', len(leaves), 'roots:', leaves[0])
    print('There are', len(dG.nodes()), 'terms')
    print('There are', len(connected_subG_list), 'connected componenets')

    if len(leaves) > 1:
        print('There are more than 1 root of ontology. Please use only one root.')
        sys.exit(1)
    if len(connected_subG_list) > 1:
        print( 'There are more than connected components. Please connect them.')
        sys.exit(1)

    return dG, leaves[0], term_size_map, term_direct_gene_map


def load_train_data(file_name, cell2id, drug2id):
    feature = []
    label = []
    feature_dict = {}
    with open(file_name, 'r') as fi:
        for line in fi:
            tokens = line.strip().split('\t')
            keys = list(cell2id.keys())[list(cell2id.values()).index(cell2id[tokens[0]])] + ";" + list(drug2id.keys())[list(drug2id.values()).index(drug2id[tokens[1]])]
            feature.append([cell2id[tokens[0]], drug2id[tokens[1]]])
            feature_dict[keys] = [cell2id[tokens[0]], drug2id[tokens[1]]]
            label.append([float(tokens[2])])
    return feature, label, feature_dict


def load_mapping(some_file):
    mapping = {}
    with  open(some_file) as fin:
        for line in fin:
            line = line.rstrip().split()
            mapping[line[1]] = int(line[0])
    return mapping

def check_file(some_file):
    if os.path.isfile(some_file):
        print(some_file)
    else:
        print('{0} file does not exist'.format(some_file))
        exit()
        
def prepare_predict_data(test_file, cell2id_mapping_file, drug2id_mapping_file):
    cell2id_mapping = load_mapping(cell2id_mapping_file)
    drug2id_mapping = load_mapping(drug2id_mapping_file)
    test_feature, test_label, feature_dict = load_train_data(test_file, cell2id_mapping, drug2id_mapping)
#    test_feature = test_feature_dict.values()
    torch_test_feature = torch.Tensor(test_feature)
    torch_test_label = torch.Tensor(test_label)
    print('Total number of cell lines = %d' % len(cell2id_mapping))
    print('Total number of drugs = %d' % len(drug2id_mapping))
    return torch_test_feature, torch_test_label, feature_dict

def prepare_train_data(train_file, test_file, cell2id_mapping_file, drug2id_mapping_file):
    print(train_file, test_file, cell2id_mapping_file, drug2id_mapping_file)
    # load mapping files
    cell2id_mapping = load_mapping(cell2id_mapping_file)
    drug2id_mapping = load_mapping(drug2id_mapping_file)
  
    train_feature, train_label, feature_dict  = load_train_data(train_file, cell2id_mapping, drug2id_mapping)
    test_feature, test_label, feature_dict  = load_train_data(test_file, cell2id_mapping, drug2id_mapping)

    print('Total number of cell lines = %d' % len(cell2id_mapping))
    print('Total number of drugs = %d' % len(drug2id_mapping))
    return (torch.Tensor(train_feature), 
            torch.FloatTensor(train_label), 
            torch.Tensor(test_feature), 
            torch.FloatTensor(test_label)), feature_dict, cell2id_mapping, drug2id_mapping

def build_input_vector(input_data, cell_features, drug_features):
    genedim = len(cell_features[0,:])
    drugdim = len(drug_features[0,:])
    print(genedim)
    print(drugdim)
    feature = np.zeros((input_data.size()[0], (genedim+drugdim)))
    #print(input_data)
    print(input_data.size()[0])
    #print(drug_features)

    for i in range(input_data.size()[0]):
        #print(int(input_data[i,0]))
        try:
            feature[i] = np.concatenate((cell_features[int(input_data[i,0])], 
                                         drug_features[int(input_data[i,1])]), axis=None)
        except IndexError:
            pass

    feature = torch.from_numpy(feature).float()
    return feature

def pearsonr(x, y):
    """Compute Pearson correlation.
    Args:
        x (torch.Tensor): 1D vector
        y (torch.Tensor): 1D vector of the same size as y.
    Raises:
        TypeError: not torch.Tensors.
        ValueError: not same shape or at least length 2.
    Returns:
        Pearson correlation coefficient.
    """
    if not isinstance(x, torch.Tensor) or not isinstance(y, torch.Tensor):
        raise TypeError('Function expects torch Tensors.')

    if len(x.shape) > 1 or len(y.shape) > 1:
        raise ValueError(' x and y must be 1D Tensors.')

    if len(x) != len(y):
        raise ValueError('x and y must have the same length.')

    if len(x) < 2:
        raise ValueError('x and y must have length at least 2.')

    # If an input is constant, the correlation coefficient is not defined.
    if bool((x == x[0]).all()) or bool((y == y[0]).all()):
        raise ValueError('Constant input, r is not defined.')

    mx = x - torch.mean(x)
    my = y - torch.mean(y)
    cost = (
        torch.sum(mx * my) /
        (torch.sqrt(torch.sum(mx**2)) * torch.sqrt(torch.sum(my**2)))
    )
    return torch.clamp(cost, min=-1.0, max=1.0)

def correlation_coefficient_loss(labels, predictions):
    """Compute loss based on Pearson correlation.
    Args:
        labels (torch.Tensor): reference values
        predictions (torch.Tensor): predicted values
    Returns:
        torch.Tensor: A loss that when minimized forces high squared correlation coefficient:
        \$1 - r(labels, predictions)^2\$  # noqa
    """
    return 1 - pearsonr(labels, predictions)**2


def mse_cc_loss(labels, predictions):
    """Compute loss based on MSE and Pearson correlation.
    The main assumption is that MSE lies in [0,1] range, i.e.: range is
    comparable with Pearson correlation-based loss.
    Args:
        labels (torch.Tensor): reference values
        predictions (torch.Tensor): predicted values
    Returns:
        torch.Tensor: A loss that computes the following:
        \$mse(labels, predictions) + 1 - r(labels, predictions)^2\$  # noqa
    """
    mse_loss_fn = nn.MSELoss()
    mse_loss = mse_loss_fn(predictions, labels)
    cc_loss = correlation_coefficient_loss(labels, predictions)
    return mse_loss + cc_loss

## DRUGCELL MODEL

In [53]:
class drugcell_nn(nn.Module):

    def __init__(self, term_size_map, term_direct_gene_map, dG, ngene, ndrug, root,
                 num_hiddens_genotype, num_hiddens_drug, num_hiddens_final):

        super(drugcell_nn, self).__init__()

        self.root = root
        self.num_hiddens_genotype = num_hiddens_genotype
        self.num_hiddens_drug = num_hiddens_drug

        # dictionary from terms to genes directly annotated with the term
        self.term_direct_gene_map = term_direct_gene_map

        # calculate the number of values in a state (term): term_size_map is the number of all genes annotated with the term
        self.cal_term_dim(term_size_map)

        # ngenes, gene_dim are the number of all genes
        self.gene_dim = ngene
        self.drug_dim = ndrug

        # add modules for neural networks to process genotypes
        self.contruct_direct_gene_layer()
        self.construct_NN_graph(dG)

        # add modules for neural networks to process drugs
        self.construct_NN_drug()

        # add modules for final layer
        final_input_size = num_hiddens_genotype + num_hiddens_drug[-1]
        self.add_module('final_linear_layer', nn.Linear(final_input_size, num_hiddens_final))
        self.add_module('final_batchnorm_layer', nn.BatchNorm1d(num_hiddens_final))
        self.add_module('final_aux_linear_layer', nn.Linear(num_hiddens_final,1))
        self.add_module('final_linear_layer_output', nn.Linear(1, 1))

    # calculate the number of values in a state (term)
    def cal_term_dim(self, term_size_map):

        self.term_dim_map = {}

        for term, term_size in term_size_map.items():
            num_output = self.num_hiddens_genotype

            # log the number of hidden variables per each term
            num_output = int(num_output)
#            print("term\t%s\tterm_size\t%d\tnum_hiddens\t%d" % (term, term_size, num_output))
            self.term_dim_map[term] = num_output


    # build a layer for forwarding gene that are directly annotated with the term
    def contruct_direct_gene_layer(self):

        for term, gene_set in self.term_direct_gene_map.items():
            if len(gene_set) == 0:
                print('There are no directed asscoiated genes for', term)
                sys.exit(1)

            # if there are some genes directly annotated with the term, add a layer taking in all genes and forwarding out only those genes
            self.add_module(term+'_direct_gene_layer', nn.Linear(self.gene_dim, len(gene_set)))


    # add modules for fully connected neural networks for drug processing
    def construct_NN_drug(self):
        input_size = self.drug_dim

        for i in range(len(self.num_hiddens_drug)):
            self.add_module('drug_linear_layer_' + str(i+1), nn.Linear(input_size, self.num_hiddens_drug[i]))
            self.add_module('drug_batchnorm_layer_' + str(i+1), nn.BatchNorm1d(self.num_hiddens_drug[i]))
            self.add_module('drug_aux_linear_layer1_' + str(i+1), nn.Linear(self.num_hiddens_drug[i],1))
            self.add_module('drug_aux_linear_layer2_' + str(i+1), nn.Linear(1,1))

            input_size = self.num_hiddens_drug[i]


    # start from bottom (leaves), and start building a neural network using the given ontology
    # adding modules --- the modules are not connected yet
    def construct_NN_graph(self, dG):

        self.term_layer_list = []   # term_layer_list stores the built neural network
        self.term_neighbor_map = {}

        # term_neighbor_map records all children of each term
        for term in dG.nodes():
            self.term_neighbor_map[term] = []
            for child in dG.neighbors(term):
                self.term_neighbor_map[term].append(child)

        while True:
            leaves = [n for n in dG.nodes() if dG.out_degree(n) == 0]
            #leaves = [n for n,d in dG.out_degree().items() if d==0]
            #leaves = [n for n,d in dG.out_degree() if d==0]

            if len(leaves) == 0:
                break

            self.term_layer_list.append(leaves)

            for term in leaves:

                # input size will be #chilren + #genes directly annotated by the term
                input_size = 0

                for child in self.term_neighbor_map[term]:
                    input_size += self.term_dim_map[child]

                if term in self.term_direct_gene_map:
                    input_size += len(self.term_direct_gene_map[term])

                # term_hidden is the number of the hidden variables in each state
                term_hidden = self.term_dim_map[term]

                self.add_module(term+'_linear_layer', nn.Linear(input_size, term_hidden))
                self.add_module(term+'_batchnorm_layer', nn.BatchNorm1d(term_hidden))
                self.add_module(term+'_aux_linear_layer1', nn.Linear(term_hidden,1))
                self.add_module(term+'_aux_linear_layer2', nn.Linear(1,1))

            dG.remove_nodes_from(leaves)


    # definition of forward function
    def forward(self, x):
        gene_input = x.narrow(1, 0, self.gene_dim)
        drug_input = x.narrow(1, self.gene_dim, self.drug_dim)

        # define forward function for genotype dcell #############################################
        term_gene_out_map = {}

        for term, _ in self.term_direct_gene_map.items():
            term_gene_out_map[term] = self._modules[term + '_direct_gene_layer'](gene_input)

        term_NN_out_map = {}
        aux_out_map = {}

        for i, layer in enumerate(self.term_layer_list):

            for term in layer:

                child_input_list = []

                for child in self.term_neighbor_map[term]:
                    child_input_list.append(term_NN_out_map[child])

                if term in self.term_direct_gene_map:
                    child_input_list.append(term_gene_out_map[term])

                child_input = torch.cat(child_input_list,1)

                term_NN_out = self._modules[term+'_linear_layer'](child_input)

                Tanh_out = torch.tanh(term_NN_out)
                term_NN_out_map[term] = self._modules[term+'_batchnorm_layer'](Tanh_out)
                aux_layer1_out = torch.tanh(self._modules[term+'_aux_linear_layer1'](term_NN_out_map[term]))
                aux_out_map[term] = self._modules[term+'_aux_linear_layer2'](aux_layer1_out)

        # define forward function for drug dcell #################################################
        drug_out = drug_input

        for i in range(1, len(self.num_hiddens_drug)+1, 1):
            drug_out = self._modules['drug_batchnorm_layer_'+str(i)]( torch.tanh(self._modules['drug_linear_layer_' + str(i)](drug_out)))
            term_NN_out_map['drug_'+str(i)] = drug_out

            aux_layer1_out = torch.tanh(self._modules['drug_aux_linear_layer1_'+str(i)](drug_out))
            aux_out_map['drug_'+str(i)] = self._modules['drug_aux_linear_layer2_'+str(i)](aux_layer1_out)

        # connect two neural networks at the top #################################################
        final_input = torch.cat((term_NN_out_map[self.root], drug_out), 1)

        out = self._modules['final_batchnorm_layer'](torch.tanh(self._modules['final_linear_layer'](final_input)))
        term_NN_out_map['final'] = out

        aux_layer_out = torch.tanh(self._modules['final_aux_linear_layer'](out))
        aux_out_map['final'] = self._modules['final_linear_layer_output'](aux_layer_out)

        return aux_out_map, term_NN_out_map
    

## TRAINING

In [54]:
def create_term_mask(term_direct_gene_map, gene_dim, cuda):
    term_mask_map = {}
    for term, gene_set in term_direct_gene_map.items():
        mask = torch.zeros(len(gene_set), gene_dim)
        for i, gene_id in enumerate(gene_set):
            mask[i, gene_id] = 1
            mask_gpu = torch.autograd.Variable(mask.cuda(cuda))
            term_mask_map[term] = mask_gpu
    return term_mask_map

In [55]:
def train_model(root, term_size_map, term_direct_gene_map, dG, train_data, 
                gene_dim, drug_dim, model_save_folder, train_epochs, batch_size, 
                learning_rate, num_hiddens_genotype, num_hiddens_drug, num_hiddens_final, 
                cell_features, drug_features):
    t = time()
    epoch_start_time = time()
    best_model = 0
    max_corr = 0
    logger = logging.getLogger(f'{model_name}')
    save_top_model = os.path.join(model_save_folder, 'results/drugcell_{}_{}.pt')
    model = drugcell_nn(term_size_map, term_direct_gene_map, dG, num_genes,
                        drug_dim, root, num_hiddens_genotype, num_hiddens_drug, num_hiddens_final)
    train_feature, train_label, test_feature, test_label = train_data

    device = torch.device("cuda")
    model.to(device)
    model.cuda(CUDA_ID)
    
    train_label_gpu = torch.autograd.Variable(train_label.cuda(CUDA_ID))
    test_label_gpu = torch.autograd.Variable(test_label.cuda(CUDA_ID))
    term_mask_map = create_term_mask(model.term_direct_gene_map, num_genes, CUDA_ID)

    # Define optimizer
    optimizer_dict = {"adam": "optim.adam"}
    #optim_value = optimizer
    optimizer = optim.Adam(model.parameters(),
                           lr=learning_rate,
                           betas=(0.9, 0.99),
                           weight_decay=1e-5,
                           eps=eps)
    optimizer.zero_grad()
    scores = {}
    epoch_list = []
    train_loss_list = []
    train_corr_list = []
    train_scc_list = []
    test_loss_list = []
    test_corr_list = []
    test_scc_list = []
    for name, param in model.named_parameters():
        term_name = name.split('_')[0]
        if '_direct_gene_layer.weight' in name:
            param.data = torch.mul(param.data, term_mask_map[term_name]) * 0.1
        else:
            param.data = param.data * 0.1

    train_loader = du.DataLoader(du.TensorDataset(train_feature,train_label),
                                 batch_size=batch_size, shuffle=False)

    test_loader = du.DataLoader(du.TensorDataset(test_feature,test_label),
                                batch_size=batch_size, shuffle=False)

    for epoch in range(epochs):
        model.train()
        epoch_list.append(epoch)
        train_predict =  torch.zeros(0,0).cuda(CUDA_ID)
        #logger.info(f"== Epoch [{epoch}/epochs}] ==")
        train_loss_mean = 0
        t = time()    
        for i, (inputdata, labels) in enumerate(train_loader):
            features = build_input_vector(inputdata, cell_features, drug_features)
            cuda_features = torch.autograd.Variable(features.cuda(CUDA_ID))
            cuda_labels = torch.autograd.Variable(labels.cuda(CUDA_ID))

            optimizer.zero_grad()
            aux_out_map, _ = model(cuda_features)

            if train_predict.size()[0] == 0:
                train_predict = aux_out_map['final'].data
            else:
                train_predict = torch.cat([train_predict, aux_out_map['final'].data], dim=0)

            train_loss = 0
            count = 0
            for name, output in aux_out_map.items():
                count +=1
                loss = nn.MSELoss()
                if name == 'final':
                    train_loss += loss(output, cuda_labels)
                else:
                    train_loss += 0.2 * loss(output, cuda_labels)
            train_loss.backward()
#            train_loss_mean = train_loss/count
            train_loss_mean = train_loss
            for name, param in model.named_parameters():
                if '_direct_gene_layer.weight' not in name:
                    continue
                term_name = name.split('_')[0]
                param.grad.data = torch.mul(param.grad.data, term_mask_map[term_name])

            optimizer.step()
        #epoch_train_test_df['train_loss'] = train_loss_mean.cpu().detach().numpy()/len(train_loader)
        train_loss_list.append(train_loss_mean.cpu().detach().numpy()/len(train_loader))
        logger.info(
            "\t **** TRAINING ****   "
            f"Epoch [{epoch + 1}/{epochs}], "
            f"loss: {train_loss_mean / len(train_loader):.5f}. "
            f"This took {time() - t:.1f} secs."
        )

        train_corr = pearson_corr(train_predict, train_label_gpu)
        train_corr_list.append(train_corr.cpu().detach().numpy())
        torch.save(model, model_save_folder + '/model_' + str(epoch) + '.pt')
        train_predictions = np.array([p.cpu() for preds in train_predict for p in preds],dtype = np.float)
        train_predictions = train_predictions[0:len(train_predictions)]
        train_labels = np.array([l.cpu() for label in train_label_gpu for l in label],dtype = np.float)
        train_scc = spearmanr(train_labels, train_predictions)[0]
        train_scc_list.append(train_scc)
        model.eval()

        test_predict = torch.zeros(0,0).cuda(CUDA_ID)

        test_loss = 0
        tissue = []
        drug = []
        for i, (inputdata, labels) in enumerate(test_loader):
            features = build_input_vector(inputdata, cell_features, drug_features)
            cuda_features = Variable(features.cuda(CUDA_ID))
            aux_out_map, _ = model(cuda_features)
            values = inputdata.cpu().detach().numpy().tolist()
            keys = [i for i in feature_dict for x in values if feature_dict [i]== x ]
            tissue = [i.split(';')[0] for i in keys]
            drug = [i.split(';')[1] for i in keys]
            loss = nn.MSELoss()
            if test_predict.size()[0] == 0:
                test_predict = aux_out_map['final'].data
                loss_a =  loss(test_predict, cuda_labels)
                test_loss += loss_a.item() 
            else:
                test_predict = torch.cat([test_predict, aux_out_map['final'].data], dim=0)
                loss_a =  loss(test_predict, cuda_labels)
                test_loss += loss_a.item()
        logger.info(
            "\t **** TEST ****   "
            f"Epoch [{epoch + 1}/{epochs}], "
            f"loss: {test_loss / len(test_loader):.5f}. "
            f"This took {time() - t:.1f} secs."
        )
        predictions = np.array([p.cpu() for preds in test_predict for p in preds] ,dtype = np.float )
        predictions = predictions[0:len(predictions)]
        labels = np.array([l.cpu() for label in labels for l in label],dtype = np.float)
        labels = labels[0:len(labels)]
        test_pearson_a = calc_pcc(torch.Tensor(predictions), torch.Tensor(labels))
        test_spearman_a = spearmanr(labels, predictions)[0]
        test_mean_absolute_error = sklearn.metrics.mean_absolute_error(y_true=labels, y_pred=predictions)
        test_r2 = sklearn.metrics.r2_score(y_true=labels, y_pred=predictions)
        test_rmse_a = np.sqrt(np.mean((predictions - labels)**2))
        test_loss_a = test_loss / len(test_loader)
        epoch_end_time = time()
        test_loss_a = test_loss/len(test_loader)
#        test_loss_a = test_loss.cpu().detach().numpy()/len(test_loader)
        test_loss_list.append(test_loss_a)
        test_corr_list.append(test_pearson_a.cpu().detach().numpy())
        test_scc_list.append(test_spearman_a)
        if epoch == 0:
            min_test_loss = test_loss_a
            scores['test_loss'] = min_test_loss
            scores['test_pcc'] = test_pearson_a.cpu().detach().numpy().tolist()
            scores['test_MSE'] = test_mean_absolute_error
            scores['test_r2'] = test_r2
            scores['test_scc'] = test_spearman_a
        if test_loss_a < min_test_loss:
            min_test_loss = test_loss_a
            scores['test_loss'] = min_test_loss
            scores['test_pcc'] = test_pearson_a.cpu().detach().numpy().tolist()
            scores['test_MSE'] = test_mean_absolute_error
            scores['test_r2'] = test_r2
            scores['test_scc'] = test_spearman_a

        if test_spearman_a >= max_corr:
            max_corr = test_spearman_a
            best_model = epoch
            pred = pd.DataFrame({"Tissue": tissue, "Drug": drug, "True": labels, "Pred": predictions}).reset_index()
            fname='/results/test_pred_' + iteration +".csv"
            pred_fname = str(model_save_folder+ "/" + fname)
            pred.to_csv(pred_fname, index=False)
#        print("epoch\t%d\tcuda_id\t%d\ttrain_corr\t%.6f\tval_corr\t%.6f\ttrain_loss\t%.6f\telapsed_time\t%s" % (epoch,
#                                                                                                                CUDA_ID,
#                                                                                                                train_corr, test_corr,
#                                                                                                                train_loss, epoch_end_time-epoch_start_time))
        epoch_start_time = epoch_end_time
        #ckpt.ckpt_epoch(epoch, test_loss_a)
    torch.save(model, model_save_folder + '/model_final.pt') 
    gc.collect()
    torch.cuda.empty_cache()
    print("Best performed model (epoch)\t%d" % best_model)
#    torch.save(save_top_model.format('epoch', '0', best_model))
    cols = ['epoch', 'train_loss', 'train_corr', 'train_scc', 'test_loss', 'test_corr', 'test_scc']
    epoch_train_test_df = pd.DataFrame(columns=cols, index=range(epochs))
    epoch_train_test_df['epoch'] = epoch_list
    epoch_train_test_df['train_loss'] = train_loss_list
    epoch_train_test_df['train_corr'] = train_corr_list
    epoch_train_test_df['train_scc'] = train_scc_list    
    epoch_train_test_df['test_loss'] = test_loss_list
    epoch_train_test_df['test_corr'] = test_corr_list
    epoch_train_test_df['test_scc'] = test_scc_list
    l_fname = 'NOTEBOOK/results/train_val_loss_results' + iteration +".csv"
    loss_results_name = str(model_save_folder+'/' + l_fname)
    epoch_train_test_df.to_csv(loss_results_name, index=False)
    print(scores)
    return scores

## INFERENCE

In [56]:
def predict_dcell(predict_data, gene_dim, drug_dim, model_file, hidden_folder,
                  batch_size, result_file, cell_features, drug_features, CUDA_ID,output_dir):
    feature_dim = gene_dim + drug_dim
    device = torch.device("cuda")
    model = torch.load(model_file, map_location='cuda:%d' % CUDA_ID)
    #checkpoint = torch.load(trained_model, map_location='cuda:%d' % CUDA_ID)
    model.to(device)
    predict_feature, predict_label, feature_dict = predict_data

    predict_label_gpu = predict_label.cuda(CUDA_ID)
    model.cuda(CUDA_ID)
    model.eval()

    test_loader = du.DataLoader(du.TensorDataset(predict_feature,predict_label), 
                                batch_size=batch_size, shuffle=False)
    model_dir = output_dir

    #Test
    test_predict = torch.zeros(0,0).cuda(CUDA_ID)
    term_hidden_map = {}
    test_loss = 0
    batch_num = 0
    test_loss_list = []
    test_corr_list = []
    drug_list = []
    tissue_list = []
    print("Begin test evaluation")
    for i, (inputdata, labels) in enumerate(test_loader):
        # Convert torch tensor to Variable
        cuda_labels = torch.autograd.Variable(labels.cuda(CUDA_ID))
        features = build_input_vector(inputdata, cell_features, drug_features)
        cuda_features = Variable(features.cuda(CUDA_ID), requires_grad=False)
        loss = nn.MSELoss()
        values = inputdata.cpu().detach().numpy().tolist()
        keys = [i for i in feature_dict for x in values if feature_dict [i]== x ]
        tissue = [i.split(';')[0] for i in keys]
        tissue_list.append(tissue)
        drug = [i.split(';')[1] for i in keys]
        drug_list.append(drug)
        # make prediction for test data
        aux_out_map, term_hidden_map = model(cuda_features)
        if test_predict.size()[0] == 0:
            test_predict = aux_out_map['final'].data
            loss_a =  loss(test_predict, cuda_labels)
            print(loss_a)
            test_loss += loss_a.item()
        else:
            test_predict = torch.cat([test_predict, aux_out_map['final'].data], dim=0)
            loss_a =  loss(test_predict, cuda_labels)
            print(loss_a)
            test_loss += loss_a.item()

        batch_num += 1
    predictions = np.array([p.cpu() for preds in test_predict for p in preds] ,dtype = np.float )
    predictions = predictions[0:len(predictions)]
    labels = np.array([l.cpu() for label in labels for l in label],dtype = np.float)
    labels = labels[0:len(labels)]
    test_pearson_a = pearson_corr(torch.Tensor(predictions), torch.Tensor(labels))
    test_spearman_a = spearmanr(labels, predictions)[0]
    test_mean_absolute_error = sklearn.metrics.mean_absolute_error(y_true=labels, y_pred=predictions)
    test_r2 = sklearn.metrics.r2_score(y_true=labels, y_pred=predictions)
    test_rmse_a = np.sqrt(np.mean((predictions - labels)**2))
    test_loss_a = test_loss / len(test_loader)
    epoch_end_time = time()
    test_loss_a = test_loss/len(test_loader)
    test_loss_list.append(test_loss_a)
    test_corr_list.append(test_pearson_a.cpu().detach().numpy())
    min_test_loss = test_loss_a
    scores = {}
    scores['test_loss'] = min_test_loss
    scores['test_pcc'] = test_pearson_a.cpu().detach().numpy().tolist()
    scores['test_MSE'] = test_mean_absolute_error
    scores['test_r2'] = test_r2
    scores['test_scc'] = test_spearman_a
    test_corr = pearson_corr(test_predict, predict_label_gpu)
    print("Test pearson corr\t%s\t%.6f" % (model.root, test_corr))
    print("Test spearman corr\t%s\t%.6f" % (model.root, test_spearman_a))
    print(scores)
    cols = ['drug', 'tissue', 'test_loss', 'test_corr', 'test_scc']
    metrics_test_df = pd.DataFrame(columns=cols, index=range(len(test_loader)))
    metrics_test_df['test_loss'] = test_loss_list
    metrics_test_df['test_corr'] = test_corr_list
    
    lr_name = 'NOTEBOOK/results/test_metrics_results' + iteration + '.csv'
    loss_results_name = str(model_dir+'/' + lr_name)
    metrics_test_df.to_csv(loss_results_name, index=False)
    np.savetxt(result_file+'/drugcell.predict', test_predict.cpu().numpy(),'%.4e')

## RUN TRAIN

In [57]:
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
#print(train_data_file)
train_data, feature_dict, cell2id_mapping, drug2id_mapping = prepare_train_data(train_data_file, 
                                                                  test_data_file, cell2id, drug2id)
gene2id_mapping = load_mapping(gene2id)
modeldir = output_dir
cell_features = np.genfromtxt(genotype, delimiter=',')
drug_features = np.genfromtxt(fingerprint, delimiter=',')

num_cells = len(cell2id_mapping)
num_drugs = len(drug2id_mapping)
num_genes = len(gene2id_mapping)
drug_dim = len(drug_features[0,:])

# load ontology
dG, root, term_size_map, term_direct_gene_map = load_ontology(onto, gene2id_mapping)

# load the number of hiddens #######
num_hiddens_genotype = genotype_hiddens

num_hiddens_drug = list(map(int, drug_hiddens.split(',')))

num_hiddens_final = final_hiddens
#####################################

CUDA_ID = CUDA_ID

train_model(root, term_size_map, term_direct_gene_map, dG, 
            train_data, num_genes, drug_dim, modeldir, epochs,
            batch_size, learning_rate, num_hiddens_genotype, num_hiddens_drug, 
            num_hiddens_final, cell_features, drug_features)
#term_size_map

Data_Curation_final/Curated_CCLE_Multiomics_files/DrugCell_ANL_inputs/temp_train.txt Data_Curation_final/Curated_CCLE_Multiomics_files/DrugCell_ANL_inputs/temp_test.txt Data_Curation_final/Curated_CCLE_Multiomics_files/DrugCell_ANL_inputs/cell2ind.txt Data_Curation_final/Curated_CCLE_Multiomics_files/DrugCell_ANL_inputs/drug2ind.txt
Total number of cell lines = 1024
Total number of drugs = 22429
There are 2981 genes
['GO:0008150']
There are 1 roots: GO:0008150
There are 2086 terms
There are 1 connected componenets
18739
1024
300


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 31.74 GiB total capacity; 29.74 GiB already allocated; 13.12 MiB free; 30.40 GiB reserved in total by PyTorch)

## RESULTS between training and validation

In [ ]:
#train_val_loss_df = pd.read_csv(lr_name)
#train_val_loss_df

## Plots

### Training loss curve

In [ ]:
## Plot of Epoch train loss
sns.set(style='whitegrid')
sns.lineplot(x="epoch",
                y="train_loss",
                data=train_val_loss_df)


## validation loss curve

In [ ]:


sns.set(style='whitegrid')
sns.lineplot(x="epoch",
                y="test_loss",
                data=train_val_loss_df)
plt.ylabel('val_loss')


## Train Pearsons CC

In [ ]:
sns.set(style='whitegrid')
sns.lineplot(x="epoch",
                y="train_corr",
                data=train_val_loss_df)
plt.ylabel('Train Pearsons CC')
#train_val_loss_df


## Validation Pearsons CC

In [ ]:
sns.set(style='whitegrid')
sns.lineplot(x="epoch",
                y="test_corr",
                data=train_val_loss_df)
plt.ylabel('validation Pearsons CC')


## Train spearmen rank C

In [ ]:
sns.set(style='whitegrid')
sns.lineplot(x="epoch",
                y="train_scc",
                data=train_val_loss_df)
plt.ylabel('Train spearmen rank C')
#train_val_loss_df

## Validation spearmen rank C

In [ ]:
sns.set(style='whitegrid')
sns.lineplot(x="epoch",
                y="test_scc",
                data=train_val_loss_df)
plt.ylabel('validation spearmen rank C')
#train_val_loss_df

In [ ]:
mean_train_pcc = train_val_loss_df.train_corr.mean()
print("Train mean pearson corr \t%.6f" % (mean_train_pcc))

In [ ]:
mean_test_pcc = train_val_loss_df.test_corr.mean()
print("Test mean pearson corr \t%.6f" % (mean_test_pcc))


In [ ]:
mean_train_scc = train_val_loss_df.train_scc.mean()

print("Test train spearmen corr \t%.6f" % (mean_train_scc))


In [ ]:
mean_test_scc = train_val_loss_df.test_scc.mean()
print("Test test spearmen corr \t%.6f" % (mean_test_scc))

## RUN INFERENCE

In [ ]:
cell_features = np.genfromtxt(genotype, delimiter=',')
drug_features = np.genfromtxt(fingerprint, delimiter=',')
num_cells = len(cell2id)
num_drugs = len(drug2id)
num_genes = len(gene2id)
drug_dim = len(drug_features[0,:])
#output_dir = params['output_dir']
model_trained_path = 'NOTEBOOK/model_final.pt'
predict_data = prepare_predict_data(test_data_file, cell2id, drug2id)


## INFERENCE RESULT FROM TRAINED RESULT

In [ ]:
predict_dcell(predict_data, num_genes, drug_dim, model_trained_path, hidden, batch_size,
            result, cell_features, drug_features, CUDA_ID, output_dir)

## INFERENCE RESULT FROM PRE-TRAINED MODEL FROM IDEKARLAB

In [ ]:
cell_features = np.genfromtxt(genotype, delimiter=',')
drug_features = np.genfromtxt(fingerprint, delimiter=',')
num_cells = len(cell2id)
num_drugs = len(drug2id)
num_genes = len(gene2id)
drug_dim = len(drug_features[0,:])
#output_dir = params['output_dir']
model_trained_path = 'Data/drugcell_v1.pt'
predict_data = prepare_predict_data(test_data_file, cell2id, drug2id)
predict_dcell(predict_data, num_genes, drug_dim, model_trained_path, hidden, batch_size,
            result, cell_features, drug_features, CUDA_ID, output_dir)

## combine result data

In [ ]:
test_df = pd.read_csv(test_data_file, sep='\t', header=None)
columns = ['CELL_LINE', "DRUG", "RESPONSE"]
test_df.columns = columns

trained_model_results_df = pd.read_csv("NOTEBOOK/MODEL/Result/trained_model_drugcell.predict", header=None)
trained_model_results_df.columns = ['TRAINED_MODEL_RESPONSE']
pre_built_model_results_df = pd.read_csv("NOTEBOOK/MODEL/Result/prebuilt_model_drugcell.predict", header=None)
pre_built_model_results_df.columns = ['PREBUILT_MODEL_RESPONSE']
#pre_built_model_results_df
test_df['TRAINED_MODEL_RESPONSE'] = trained_model_results_df['TRAINED_MODEL_RESPONSE']
test_df['PREBUILT_MODEL_RESPONSE'] =pre_built_model_results_df['PREBUILT_MODEL_RESPONSE']
test_name = "NOTEBOOK/MODEL/Result/DrugCell_test_data.prediction_" + iteration + ".csv"
test_df.to_csv(test_name, index=False)